

```
Go to the directory of your dataset which contains the images and csv/txt file for images.
Example : Here dataset present into the handwritten_digit_dataset
/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset
```



In [1]:
%cd /content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset

/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset


```
Clone the paddleocr folder in your dataset directory[only first time required to clone]
Example : In above case git clone inside the handwritten_digit_dataset
Version of the git clone is 2.7.5.
THis clone the all required documents in PaddleOCR folder.
```


In [ ]:
# !git clone https://github.com/kuldipparaliya/paddleOCR_2.7.5.git

```
Go to the PaddleOCR Folder in which clone the all documents.
```

In [ ]:
%cd PaddleOCR_2.7.5

/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset/PaddleOCR


```
Install all the library present inside the requirements.txt file into the PaddleOCR Folder.
```

In [ ]:
!pip install -r requirements.txt

      Successfully uninstalled opencv-python-4.8.0.76
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.8.0.76
    Uninstalling opencv-contrib-python-4.8.0.76:
      Successfully uninstalled opencv-contrib-python-4.8.0.76


```
Restart the session and go to the clone directory in our case
Example : handwritten_digit_dataset/PaddleOCR
```

```
PaddleOCR required the label file in particular format and this format must be create by the PaddleOCR itself.
Below script will automatically generate the particular format.

!python ppocr/utils/gen_label.py --mode="rec" --input_path="input_path" --output_label="generated label file name[output_label.txt]"


input_path : path of the csv/txt file which contains the image_name and text present into that image.
--------Format of csv/txt file---------
image1.jpg, text1
image2.jpg, text2
---------------------------------------

In image name extension must be required.
image name and text must be separated by (,).
If input_file is csv file then remove the column name from the csv file.

If our dataset have train,test and validation separate folder then for all folder[train,test,val] we require to generate the label file.
```

In [ ]:
!python ppocr/utils/gen_label.py --mode="rec" --input_path="/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset/with_extension.csv" --output_label="output_label1.txt"

Generate rec label


```
Download the pretrained model from the official website of paddleocr.
Decompress the downloaded tar file into the original form.
```

In [ ]:
%cd PaddleOCR_2.7.5/
# Download the pre-trained model of en_PP-OCRv3
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
# Decompress model parameters
%cd pretrain_models
!tar -xf en_PP-OCRv3_rec_train.tar && rm -rf en_PP-OCRv3_rec_train.tar

```
Move Back to the PaddleOCR Folder in which our clone data available.
```

In [ ]:
%cd ..

```
In our given example present working directory is :     
handwritten_digit_dataset/PaddleOCR
```

```
Install the library of paddle for gpu use cases. Version of the paddle is 2.6.1.

!pip install paddlepaddle-gpu

Other Library require to work on the cpu.
```

In [ ]:
!pip install paddlepaddle-gpu==2.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.1 MB/s eta 0:00:00


```
Before we start fine-tuning of the model we require to update the yml file which is present in this location.
configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml

Below changes required:

epoch_num :
For Training :
data_dir : Path of the train_images
label_file_list : Path of the trainig label text file generate in above script.

For Eval:
data_dir : Path of the test_images
label_file_list : Path of the test label text file generate in above script.

If we have available the custom charcter dictionary path then update :   
character_dict_path : path of the custom dictionary
```

```
Perform the fine-tuning of the model. Below script start the epochs.
Update the yml file as per your requirement and paste the path of updated file.

!python3 -m paddle.distributed.launch  tools/train.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset/PaddleOCR/pretrain_models/en_PP-OCRv3_rec_train/best_accuracy.pdparams

Replace the path of the pretrained_model if you change as the time of installation.
```

In [ ]:
!python3 -m paddle.distributed.launch  tools/train.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset/PaddleOCR/pretrain_models/en_PP-OCRv3_rec_train/best_accuracy.pdparams

LAUNCH INFO 2024-05-03 09:32:31,181 -----------  Configuration  ----------------------
LAUNCH INFO 2024-05-03 09:32:31,181 auto_parallel_config: None
LAUNCH INFO 2024-05-03 09:32:31,181 auto_tuner_json: None
LAUNCH INFO 2024-05-03 09:32:31,181 devices: None
LAUNCH INFO 2024-05-03 09:32:31,181 elastic_level: -1
LAUNCH INFO 2024-05-03 09:32:31,181 elastic_timeout: 30
LAUNCH INFO 2024-05-03 09:32:31,181 enable_gpu_log: True
LAUNCH INFO 2024-05-03 09:32:31,181 gloo_port: 6767
LAUNCH INFO 2024-05-03 09:32:31,181 host: None
LAUNCH INFO 2024-05-03 09:32:31,181 ips: None
LAUNCH INFO 2024-05-03 09:32:31,181 job_id: default
LAUNCH INFO 2024-05-03 09:32:31,181 legacy: False
LAUNCH INFO 2024-05-03 09:32:31,181 log_dir: log
LAUNCH INFO 2024-05-03 09:32:31,181 log_level: INFO
LAUNCH INFO 2024-05-03 09:32:31,181 log_overwrite: False
LAUNCH INFO 2024-05-03 09:32:31,182 master: None
LAUNCH INFO 2024-05-03 09:32:31,182 max_restart: 3
LAUNCH INFO 2024-05-03 09:32:31,182 nnodes: 1
LAUNCH INFO 2024-05-03 0

```
Below Script will export the model into the file so that we can use for the inference
```

In [ ]:
#Conversion of Recognizer Trained weights to Inference
!python3 tools/export_model.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=output/v3_en_mobile/latest  Global.save_inference_dir=./inference/en_PP-OCRv3_rec/

```
Checking whether the paddle library is comptible with GPU or CPU
```

In [ ]:
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


```
Code for see the output for particular image
```

In [ ]:
from paddleocr import PaddleOCR,draw_ocr


```
rec_model_dir = file path where the model export.
Based on CPU or GPU specify the use_gpu = True/False.
```

In [ ]:
custom_ocr = PaddleOCR(use_angle_cls=True,
                rec_model_dir='./inference/en_PP-OCRv3_rec',
                det_model_dir='./output/det_db_inference',
                rec_char_dict_path='ppocr/utils/en_dict.txt',
                use_gpu=True,
                show_log=False,
                cls=False,
                det=True)

download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:06<00:00, 323kiB/s]


```
Specify the path of the image
```

In [ ]:
import cv2
img_path = "/content/1.jpg"
img = cv2.imread(img_path)
result = custom_ocr.ocr(img)
result

[[[[[176.0, 115.0], [220.0, 115.0], [220.0, 151.0], [176.0, 151.0]],
   ('2', 0.5624247789382935)],
  [[[563.0, 113.0], [609.0, 113.0], [609.0, 152.0], [563.0, 152.0]],
   ('(6', 0.9097020626068115)],
  [[[953.0, 116.0], [996.0, 116.0], [996.0, 151.0], [953.0, 151.0]],
   ('107', 0.6033260226249695)],
  [[[138.0, 201.0], [250.0, 201.0], [250.0, 235.0], [138.0, 235.0]],
   ('1909', 0.5509264469146729)],
  [[[527.0, 197.0], [626.0, 197.0], [626.0, 236.0], [527.0, 236.0]],
   ('11993', 0.5260910987854004)],
  [[[847.0, 195.0], [1011.0, 184.0], [1015.0, 249.0], [852.0, 259.0]],
   ('1910990', 0.8572630286216736)],
  [[[179.0, 276.0], [225.0, 276.0], [225.0, 317.0], [179.0, 317.0]],
   ('7', 0.5548692345619202)],
  [[[567.0, 278.0], [611.0, 278.0], [611.0, 314.0], [567.0, 314.0]],
   ('0', 0.8846083879470825)],
  [[[955.0, 276.0], [993.0, 276.0], [993.0, 312.0], [955.0, 312.0]],
   ('11', 0.584700882434845)],
  [[[814.0, 335.0], [1059.0, 351.0], [1054.0, 426.0], [809.0, 410.0]],
   ('119To1

Extract the only text from the output

In [ ]:
for i in range(len(result[0])):
  line_text = result[0][i][1][0]
  all_text += '\n' + line_text